In [1]:
import pickle, random, copy, platform, os, time
import numpy as np
import tensorflow as tf
import email_sending
import tflearn
from PIL import Image, ImageDraw
from __future__ import division, print_function, absolute_import

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
def read_data():
    with open('GAN', 'rb') as f:
        data = pickle.load(f)
        band1 = data[0]
        band2 = data[1]
        band1, band2 = np.reshape(band1, (-1,75,75,1)), np.reshape(band2, (-1,75,75,1))
        label = data[2]
        #reshape
    return band1, band2, label

# expresses the label data in one-hot encoding.
def onehot_encoding (Y, y_class):
    Y_onehot = np.zeros((Y.size, int(y_class)))
    for i in range(Y.size):
        Y_onehot[i][Y[i]] = 1
    return Y_onehot

In [3]:
band1, band2, totalY = read_data()
band1.shape

(1604, 75, 75, 1)

In [4]:
def resnet32(x, classes, n = 5): #classes =2
    net = tflearn.conv_2d(x, 16, 3, regularizer='L2', weight_decay=0.0001)
    print(net)
    net = tflearn.residual_block(net, n, 16)
    print(net)
    net = tflearn.residual_block(net, 1, 32, downsample=True)
    print(net)
    net = tflearn.residual_block(net, n - 1, 32)
    print(net)
    net = tflearn.residual_block(net, 1, 64, downsample=True)
    print(net)
    net = tflearn.residual_block(net, n - 1, 64)
    print(net)
    net = tflearn.batch_normalization(net)
    print(net)
    net = tflearn.activation(net, 'relu')
    print(net)
    net = tflearn.global_avg_pool(net)
    print(net)
    net = tflearn.fully_connected(net, classes, activation='softmax')
    print(net)
    return net

In [5]:
def train(num_epochs=10):
    batch_size = 100
    classes = 2

    img_prep = tflearn.ImagePreprocessing()
    img_prep.add_featurewise_zero_center()
    img_prep.add_featurewise_stdnorm()

    img_aug = tflearn.ImageAugmentation()
    img_aug.add_random_flip_leftright()
    #img_aug.add_random_rotation(max_angle=25)
    img_aug.add_random_crop([75,75], padding=4)

    Input = tflearn.input_data(shape=[None, 75, 75, 1], dtype='float', data_preprocessing=img_prep,data_augmentation=img_aug)


    band1, band2, totalY = read_data()
    band1, totalY = tflearn.data_utils.shuffle(band1, totalY)
    totalY=onehot_encoding(totalY, classes)
    X, Y = band1[:1200], totalY[:1200]
    testX, testY =band1[1200:1600], totalY[1200:1600]

    network = tflearn.regression(resnet32(Input,classes),optimizer='adam',loss='categorical_crossentropy')
    print(X.shape, Y.shape)

    model = tflearn.DNN(network,tensorboard_verbose=3)
    #checkpoint_path='/tmp'
    model.fit(X, Y, n_epoch=num_epochs, shuffle=True, validation_set=(testX, testY),
              show_metric=True, batch_size=batch_size, run_id='resnet_iceberg')

if __name__ == '__main__':
    train()

Training Step: 119  | total loss: 0.49777 | time: 32.683s
| Adam | epoch: 010 | loss: 0.49777 - acc: 0.7675 -- iter: 1100/1200
Training Step: 120  | total loss: 0.49852 | time: 36.539s
| Adam | epoch: 010 | loss: 0.49852 - acc: 0.7688 | val_loss: 0.52512 - val_acc: 0.7425 -- iter: 1200/1200
--
